In [ ]:
import os

In [1]:
directory = "results/classifications/20211102/"
suffix = '20211102'

### Rename files

In [ ]:
# for file in os.listdir(directory):
#     #print(file[0:58]+'20201215.tif')
#     os.rename(directory+file, directory+file[0:58]+'20201215.tif')

## Mosiac tifs

In [ ]:
#build vrt
os.chdir(directory)
os.system("gdalbuildvrt " + "Sahel_gm_mads_two_seasons_"+suffix+"_mosaic.vrt *.tif")

#build mosaic from .vrt
os.system("gdal_translate "\
       "-co BIGTIFF=YES "\
       "-co COMPRESS=DEFLATE "\
       "-co ZLEVEL=9 "\
       "-co PREDICTOR=1 "\
       "-co TILED=YES "\
       "-co BLOCKXSIZE=1024 "\
       "-co BLOCKYSIZE=1024 "\
       "-co NUM_THREADS=29 "\
       "--config GDAL_CACHEMAX 512 "
       + "Sahel_gm_mads_two_seasons_"+suffix+"_mosaic.vrt " + "Sahel_gm_mads_two_seasons_"+suffix+"_mosaic.tif")

## Clip mosaic to AEZ

In [2]:
import xarray as xr
import numpy as np
import geopandas as gpd
from datacube.utils.cog import write_cog

from deafrica_tools.spatial import xr_rasterize
from deafrica_tools.dask import create_local_dask_cluster

In [3]:
create_local_dask_cluster()

Client Scheduler: tcp://127.0.0.1:40259 Dashboard: /user/chad/proxy/8787/status,Cluster Workers: 1 Cores: 31 Memory: 254.70 GB


In [4]:
aez_region = 'data/Sahel.geojson'
pred_tif = "results/classifications/"+suffix+"/Sahel_gm_mads_two_seasons_"+suffix+"_mosaic.tif"
chunks={'x':10000, 'y':10000}

In [5]:
aez = gpd.read_file(aez_region)
da = xr.open_rasterio(pred_tif)

In [6]:
aez = xr_rasterize(aez, da)

/env/lib/python3.8/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [7]:
da = da.chunk(chunks).where(aez.chunk(chunks)).astype(np.int8)

In [8]:
%%time
da = da.compute()

distributed.nanny - WARNING - Restarting worker


CPU times: user 57.5 s, sys: 2min 52s, total: 3min 49s
Wall time: 6min 48s


In [9]:
write_cog(da.squeeze(),
          pred_tif[:-4]+'_clipped.tif',
          overwrite=True,
          NUM_THREADS="ALL_CPUS"
         )

PosixPath('results/classifications/20211102/Sahel_gm_mads_two_seasons_20211102_mosaic_clipped.tif')

In [ ]:
# write_cog(aez.squeeze(),
#           'data/WDPA_eastern.tif',
#           overwrite=True)